In [ ]:
import datetime

In [ ]:
def db_execute(query,data):
    print(f'OK -----   {query} + ',data)

In [ ]:
class Plc:
    def __init__(self,address):
        self.address = address
        self.connect()

    def connect(self):
        return (True,f'Connected to {self.address}')

In [ ]:
class Variable:

    def __init__(self,path,var_type,plc,
                name='',ctrl_type = 'state'):
        self.path = path
        self.name = name if name else path
        self.var_type = var_type
        self.plc = plc
        self.test = False

    @property
    def value(self):
        if self.test:
            return self.test
        else: 
            return self.plc.read_by_name(self.path,self.var_type)# read from plc


In [ ]:
class StateControler:

    QUERY = 'BASIC_QUERY'

    def __init__(self,var):
        self.var = var
        self.prev_state = self.var.value

    @property
    def change_detected(self):
        return self.var.value != self.prev_state
    
    def set_previous_state(self):
        self.prev_state = self.var.value

    def check_state(self):
        if self.change_detected:
            print('State is changed')
            self.execute_query()
            self.set_previous_state()

    def get_data(self):
        return 'Some DATA'

    def execute_query(self):
        db_execute(self.QUERY, self.get_data())


In [ ]:
class SwitchControl(StateControler):

    QUERY = 'SWITCH_QUERY'

    last_change_time = ''

    @property
    def time_of_switch(self):
        return datetime.datetime.now()

    def get_data(self):
        name = self.var.name
        value = self.var.value
        timest = self.time_of_switch

        return (name,timest,value)



        

In [30]:
import pyads
plc = pyads.Connection('192.168.1.177.1.1',801)
plc.open()

2022-04-17T15:13:54+0300 Info: Connected to 192.168.1.177


In [31]:
var_c = Variable('MAIN.bSWITCH_1',pyads.PLCTYPE_BOOL,plc, name='Switch 1')
var_f = Variable('MAIN.bSWITCH_2',pyads.PLCTYPE_BOOL,plc, name='Switch 2')
var_d = Variable('MAIN.bSWITCH_3',pyads.PLCTYPE_BOOL,plc, name='Switch 3')
var_g = Variable('MAIN.bSWITCH_4',pyads.PLCTYPE_BOOL,plc, name='Switch 4')

s1_sc = SwitchControl(var_c)
s2_sc = SwitchControl(var_f)
s3_sc = SwitchControl(var_d)
s4_sc = SwitchControl(var_g)

while True:
    s1_sc.check_state()
    s2_sc.check_state()
    s3_sc.check_state()
    s4_sc.check_state()

plc.close()



State is changed
OK -----   SWITCH_QUERY +  ('Switch 1', datetime.datetime(2022, 4, 17, 15, 14, 35, 945235), True)
State is changed
OK -----   SWITCH_QUERY +  ('Switch 1', datetime.datetime(2022, 4, 17, 15, 14, 39, 709965), False)
State is changed
OK -----   SWITCH_QUERY +  ('Switch 2', datetime.datetime(2022, 4, 17, 15, 14, 45, 412277), True)
State is changed
OK -----   SWITCH_QUERY +  ('Switch 2', datetime.datetime(2022, 4, 17, 15, 14, 46, 944414), False)
State is changed
OK -----   SWITCH_QUERY +  ('Switch 2', datetime.datetime(2022, 4, 17, 15, 14, 50, 401270), True)
State is changed
OK -----   SWITCH_QUERY +  ('Switch 2', datetime.datetime(2022, 4, 17, 15, 14, 51, 320130), False)
State is changed
OK -----   SWITCH_QUERY +  ('Switch 3', datetime.datetime(2022, 4, 17, 15, 14, 52, 92901), True)
State is changed
OK -----   SWITCH_QUERY +  ('Switch 3', datetime.datetime(2022, 4, 17, 15, 14, 52, 719899), False)
State is changed
OK -----   SWITCH_QUERY +  ('Switch 4', datetime.datetime(20

ADSError: ADSError: timeout elapsed (1861). 

2022-04-17T19:26:18+0300 Warning: InvokeId mismatch: waiting for 0x0 received 0x187c1b
2022-04-17T19:26:18+0300 Warning: No response pending


In [29]:
plc.close()

2022-04-17T15:09:24+0300 Info: connection closed by remote
